In [ ]:
import json
import time
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from bs4 import BeautifulSoup
import os

# 定义函数，执行获取过程
def fetch_links_for_user(driver, user_name, retry_count=2):
    try:
        driver.get('https://amae-koromo.sapk.ch/')
        
        # 等待页面加载并找到输入框
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'mui-3'))
        )

        input_field_1 = driver.find_element(By.ID, 'mui-3')
        input_field_1.clear()  # 清除已有内容
        input_field_1.send_keys(user_name)  # 输入用户名

        # 等待下拉菜单出现并选择第一个选项
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[id^="mui-3-option"]'))
        )
        option = driver.find_element(By.ID, 'mui-3-option-0')
        option.click()

        # 等待新页面加载，假设页面有某个特定元素才完全加载（如标题或者按钮）
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'h4'))
        )

        # 启动页面滚动以确保加载所有内容
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)  # 等待页面加载
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # 获取新页面的 HTML 内容
        new_page_html = driver.page_source

        # 使用 BeautifulSoup 解析 HTML
        soup = BeautifulSoup(new_page_html, 'html.parser')

        # 查找所有 <a> 标签
        links = soup.find_all('a', href=True)

        # 过滤出 href 以 "https://game.maj-soul.com/1/?paipu=" 开头的链接
        filtered_links = [link['href'] for link in links if link['href'].startswith('https://game.maj-soul.com/1/?paipu=')]
        return filtered_links

    except (TimeoutException, NoSuchElementException, WebDriverException) as e:
        print(f"错误：{user_name} 页面加载失败 ({str(e)})")
        if retry_count > 0:
            print(f"正在重试 {user_name} ... 剩余重试次数: {retry_count}")
            time.sleep(2)
            return fetch_links_for_user(driver, user_name, retry_count - 1)  # 递归重试
        else:
            print(f"{user_name} 已经尝试 {2 - retry_count} 次，跳过此用户")
            return []

# 读取 Excel 文件中的网络名
excel_file = '../Data/user.xlsx'  # Excel 文件路径
df = pd.read_excel(excel_file)

# 假设Excel文件中有一个 "Rname" 列包含网络名
user_names = df['Rname'].tolist()

# 获取前 N 条用户数据的函数
def get_limited_user_names(user_names, num_users):
    return user_names[:num_users] if num_users > 0 else user_names

# 获取用户数的限制（例如：读取前 10 个用户）
num_users = 500  # 修改为你希望读取的用户数
user_names = get_limited_user_names(user_names, num_users)

# 初始化 Chrome 浏览器
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 确保保存数据的文件夹存在
output_dir = '../Data/json/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 处理用户链接的函数，减少文件 I/O
def process_user_links(user_name):
    print(f"正在获取 {user_name} 的链接...")
    user_links = fetch_links_for_user(driver, user_name)
    return {user_name: user_links}

# 为每个用户执行获取链接的操作，并每100个用户存储为一个 JSON 文件
batch_size = 50  # 每100个用户存储一次
user_batch_size = 5  # 每10个用户临时保存一次

batch_links = {}
user_counter = 0  # 计数器，用于追踪当前处理的用户数
batch_counter = 1  # 批次计数器

for idx, user_name in enumerate(user_names):
    try:
        # 获取当前用户的链接
        user_links = process_user_links(user_name)
        batch_links.update(user_links)

        # 每10个用户保存一次临时数据
        if (idx + 1) % user_batch_size == 0:
            print(f"已获取并保存 {user_batch_size} 个用户的数据...")

        # 每500个用户保存一次数据到文件
        if (idx + 1) % batch_size == 0 or (idx + 1) == len(user_names):
            # 生成 JSON 文件路径
            json_file_path = os.path.join(output_dir, f'user_links_batch_{batch_counter}.json')

            # 将当前批次的数据保存到文件
            with open(json_file_path, 'w', encoding='utf-8') as json_file:
                json.dump(batch_links, json_file, ensure_ascii=False, indent=4)

            print(f"批次 {batch_counter} 的数据已保存 ({min(idx + 1, len(user_names))} 个用户)")

            # 重置临时数据结构以便下一个批次
            batch_links = {}
            batch_counter += 1  # 增加批次编号

    except Exception as e:
        print(f"处理 {user_name} 时发生错误：{e}")

# 输出结果
print(f"共获取了 {len(user_names)} 个用户的链接")

# 关闭浏览器
driver.quit()
